In [ ]:
# @infography technologies, Nepal
# for Datascience and Machine Learning

In [1]:
import pandas as pd
import numpy as np
import matplotlib as pyplot
import seaborn as sns

pd.set_option('display.max_columns',300)



In [2]:
artist_alias_df = pd.read_table("/Users/prakashchandraprasad/Documents/git-repos/music-recommendation/profiledata_06-May-2005/artist_alias.txt",header = None)

In [3]:
artist_alias_df.head()

,0,1
0,1092764.0,1000311
1,1095122.0,1000557
2,6708070.0,1007267
3,10088054.0,1042317
4,1195917.0,1042317


In [4]:
artist_alias_df.columns = ['badID','goodID']

In [5]:
artist_alias_df.head()

,badID,goodID
0,1092764.0,1000311
1,1095122.0,1000557
2,6708070.0,1007267
3,10088054.0,1042317
4,1195917.0,1042317


In [6]:
artist_alias_df.dropna(axis=0,inplace=True)
artist_alias_df['badID'] = artist_alias_df['badID'].map(lambda v :int(v))
artist_alias_df['goodID'] = artist_alias_df['goodID'].map(lambda v :int(v))

In [7]:
artist_alias_df.head()

,badID,goodID
0,1092764,1000311
1,1095122,1000557
2,6708070,1007267
3,10088054,1042317
4,1195917,1042317


In [8]:
# artist_data_df = pd.read_csv('/Users/prakashchandraprasad/Documents/git-repos/music-recommendation/profiledata_06-May-2005/artist_data.txt',delim_whitespace=True,skipinitialspace=True,error_bad_lines=False,header=None)

In [9]:
user_artist_df = pd.read_csv('/Users/prakashchandraprasad/Documents/git-repos/music-recommendation/profiledata_06-May-2005/user_artist_data.txt',delimiter=' ',header=None)

In [10]:
user_artist_df.head()

,0,1,2
0,1000002,1,55
1,1000002,1000006,33
2,1000002,1000007,8
3,1000002,1000009,144
4,1000002,1000010,314


In [11]:
user_artist_df.columns = ['userID','artistID','playcount']

In [12]:
user_artist_df.head()

,userID,artistID,playcount
0,1000002,1,55
1,1000002,1000006,33
2,1000002,1000007,8
3,1000002,1000009,144
4,1000002,1000010,314


In [13]:
user_artist_df['userID'] = user_artist_df['userID'].map(lambda v :int(v))
user_artist_df['artistID'] = user_artist_df['artistID'].map(lambda v :int(v))
user_artist_df['playcount'] = user_artist_df['playcount'].map(lambda v :int(v))


In [14]:
df1=user_artist_df.copy()
df2 = artist_alias_df.copy()

In [17]:
df3 = df1.loc[df1['artistID'].isin(df2['badID'])]

In [18]:
df3.head()

,userID,artistID,playcount
54,1000002,1000434,89
92,1000002,1000762,1
122,1000002,1001220,1
170,1000002,1001410,5
205,1000002,1002498,1


In [19]:
dict1 = dict(zip(df2['badID'],df2['goodID']))

In [22]:
for i,j in enumerate(df3.index.values.tolist()):
    df1['artistID'].iloc[j]=dict1[df1['artistID'].iloc[j]]

In [23]:
df1.isnull().sum()

userID       0
artistID     0
playcount    0
dtype: int64

In [25]:
df1.dtypes

userID       int64
artistID     int64
playcount    int64
dtype: object

In [26]:
#using pyspark to model the recommendation system

In [27]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession.builder.appName('music recommendation system').getOrCreate()

In [29]:
music_df = spark.createDataFrame(df1)

In [30]:
music_df.columns

['userID', 'artistID', 'playcount']

In [31]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [32]:
(train,test) = music_df.randomSplit([0.8,0.2])

In [33]:
als = ALS(maxIter=5, regParam=0.01, userCol="userID", itemCol="artistID", ratingCol="playcount")

In [ ]:
model = als.fit(train)